In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
np.random.seed(0)

# /home/coenraadmiddel/miniconda3/envs/venvRossmann
import os
import wget
from pathlib import Path

%autoreload 2



In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Using {}".format(DEVICE))

In [ ]:
path = '/home/coenraadmiddel/Documents/RossmannStoreSales/Input/'
trainset_name = 'rossman-store-sales'
path_train = path+'train.csv'
path_test = path+'test.csv'
path_store = path+'store.csv'

In [ ]:
store = pd.read_csv(path_store)
store.fillna(0, inplace=True)

In [ ]:
train = pd.read_csv(path_train, parse_dates=[2])
test = pd.read_csv(path_test, parse_dates=[2])
# test.fillna(value = 1, inplace = True)

In [ ]:
store

In [ ]:
# merge data with store 
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

# split the last 6 weeks data as hold-out set (idea from Gert https://www.kaggle.com/c/rossmann-store-sales/discussion/18024)
train = train.sort_values(['Date'],ascending = False)
train_total = train.copy()

split_index = 6*7*1115
valid = train[:split_index] 
train = train[split_index:]

# only use train of Sales>0 and Open is 1
valid = valid[(valid.Open != 0)&(valid.Sales >0)]
train = train[(train.Open != 0)&(train.Sales >0)]
train_total = train_total[(train_total.Open != 0)&(train_total.Sales >0)]

In [ ]:
np.log1p(train['Sales']).mean()

In [ ]:
print(train.shape)
# print(valid.shape)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# check distribution of sales in train set
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
g1 = sns.distplot(train['Sales'],hist = True,label='skewness:{:.2f}'.format(train['Sales'].skew()),ax = ax1)
g1.legend()
g1.set(xlabel = 'Sales', 

ylabel = 'Density', title = 'Sales Distribution')
g2 = sns.distplot(np.log1p(train['Sales']), hist = True,label='skewness:{:.2f}'.format(np.log1p(train['Sales']).skew()),ax=ax2)
g2.legend()
g2.set(xlabel = 'log(Sales+1)',ylabel = 'Density', title = 'log(Sales+1) Distribution')
plt.show()

In [ ]:
train.info()

In [ ]:
train.head().T

In [ ]:
train["StateHoliday"].value_counts()

In [ ]:


def process(data, isTest = False):
    # label encode some features - this does not work. fixing it below
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    # train.StoreType.replace(mappings, inplace=True)
    # train.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    
        
    # extract some features from date column  
    data['Month'] = data.Date.dt.month
    data['Year'] = data.Date.dt.year
    data['Day'] = data.Date.dt.day
    data['WeekOfYear'] = data.Date.dt.weekofyear
    
    # calculate competiter open time in months
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + \
        (data.Month - data.CompetitionOpenSinceMonth)
    data['CompetitionOpen'] = data['CompetitionOpen'].apply(lambda x: x if x > 0 else 0)
    
    # calculate promo2 open time in months
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + \
        (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['PromoOpen'] = data['PromoOpen'].apply(lambda x: x if x > 0 else 0)
                                                 
    # Indicate whether the month is in promo interval
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['month_str'] = data.Month.map(month2str)

    def check(row):
        if isinstance(row['PromoInterval'],str) and row['month_str'] in row['PromoInterval']:
            return 1
        else:
            return 0
        
    data['IsPromoMonth'] =  data.apply(lambda row: check(row),axis=1)    
    
    # select the features we need
    features = ['Store'
                , 'DayOfWeek'
                , 'Promo'
                , 'StateHoliday'
                , 'SchoolHoliday'
                , 'StoreType'
                , 'Assortment'
                , 'CompetitionDistance'
                , 'Promo2SinceWeek'
                , 'Promo2SinceYear'
                , 'Year'
                , 'Month'
                , 'Day'
                , 'WeekOfYear'
                , 'CompetitionOpen'
                , 'PromoOpen'
                , 'IsPromoMonth']  
    if not isTest:
        features.append('Sales')
      
       
    data = data[features]
    return data

In [ ]:
train = process(train)

# sort by index
train.sort_index(inplace = True)
train_total.sort_index(inplace = True)


In [ ]:
valid = process(valid)

# sort by index
valid.sort_index(inplace = True)
# train_total.sort_index(inplace = True)

categorical_columns = ['Store',
                        'DayOfWeek',
                        'Promo',
                        'StateHoliday',
                        'SchoolHoliday',
                        'StoreType',
                        'Assortment',
                        # 'Year',
                        # 'Month',
                        # 'Day',
                        # 'WeekOfYear',
                        'IsPromoMonth']
categorical_dims =  {}

for col in categorical_columns:
    print(col, valid[col].nunique())
    valid[col] = valid[col].astype(str)
    l_enc = LabelEncoder()
    # train[col] = train[col].fillna("VV_likely")
    valid[col] = l_enc.fit_transform(valid[col].values)
    # categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)
    
    
print(categorical_dims)

In [ ]:
X_valid_all, y_valid_all = valid.drop(columns = ['Sales']), np.log1p(valid[['Sales']])

In [ ]:
train.head().T

In [ ]:
train.columns

In [ ]:
    
categorical_columns = ['Store',
                        'DayOfWeek',
                        'Promo',
                        'StateHoliday',
                        'SchoolHoliday',
                        'StoreType',
                        'Assortment',
                        # 'Year',
                        # 'Month',
                        # 'Day',
                        # 'WeekOfYear',
                        'IsPromoMonth']
categorical_dims =  {}

for col in categorical_columns:
    print(col, train[col].nunique())
    train[col] = train[col].astype(str)
    l_enc = LabelEncoder()
    # train[col] = train[col].fillna("VV_likely")
    train[col] = l_enc.fit_transform(train[col].values)
    # categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)
    
    
print(categorical_dims)

In [ ]:
train.head().T.reset_index()

In [ ]:
# split x and y
X_all, y_all = train.drop(columns = ['Sales']), np.log1p(train[['Sales']])

In [ ]:
train.describe()

In [ ]:
train.head().append(train.sample(5)).append(train.tail()).T.reset_index()

In [ ]:
X_all_cats = X_all[categorical_columns]

In [ ]:
# make a list of the number of unique values in each categorical column

catmaxlist = [X_all_cats[col].nunique() for col in X_all_cats.columns]

In [ ]:
catmaxlist

In [ ]:
import random
random.seed(42)
target = 'Sales'
unused_feat = ['Set']

features = [ col for col in train.columns if col not in unused_feat+[target]] 
print('Feartures to be used', features)

cat_dims = [X_all_cats[x].nunique() for x in X_all_cats.columns]
print(cat_dims)

# define your embedding sizes : here just a random choice
cat_emb_dim =  [random.randint(2, min(x, 10)) for x in catmaxlist]
print(cat_emb_dim)
print("Categorical Dimensions: ", len(cat_dims))
print("Categorical Embedding Dimensions: ", len(cat_emb_dim))
assert len(cat_dims) == len(cat_emb_dim)

print("Number of categorical features and embedding dimensions must be equal")


In [ ]:
if "Set" not in train.columns:
    train.reset_index(inplace=True, drop=True)
    train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [ ]:
train_indices.shape

In [ ]:
X_all.values[train_indices].shape

In [ ]:
X_train = X_all.values[train_indices]
y_train = y_all.values[train_indices].reshape(-1, 1)

X_valid = X_all.values[valid_indices]
y_valid = y_all.values[valid_indices].reshape(-1, 1)

X_test = X_all.values[test_indices]
y_test = y_all.values[test_indices].reshape(-1, 1)

In [ ]:
train.head().T.reset_index()

In [ ]:
train['Day'].value_counts()

In [ ]:
#get categorical feature indexes 

cat_idxs = [X_all.columns.get_loc(c) for c in categorical_columns if c in X_all]


In [ ]:
print('Building model...')
print('Number of features: {}'.format(X_train.shape[1]))
print('Number of training samples: {}'.format(X_train.shape[0]))
print('Number of categorical features: {}'.format(len(cat_dims)))
print('Cat_dims: {}'.format(cat_dims))
print('Indexes of categorical features: {}'.format(cat_idxs))
print('List of categorical features and their embedding dimensions: {}'.format(list(zip(X_all_cats.columns, cat_emb_dim))))



wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb

sudo dpkg -i cuda-keyring_1.0-1_all.deb


In [ ]:
 torch.cuda.is_available()

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Using {}".format(DEVICE))

clf = TabNetRegressor(cat_dims=cat_dims
                      , cat_emb_dim=cat_emb_dim
                      , cat_idxs=cat_idxs
                      , device_name = DEVICE
)

In [ ]:
max_epochs = 20 if not os.getenv("CI", False) else 2
print(max_epochs)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

In [ ]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
    max_epochs=max_epochs,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    # num_workers=0,
    # drop_last=False,
    # augmentations=aug, #aug
) 

# Save model and load

In [ ]:
# save tabnet model
saving_path_name = "./tabnet_model_test_second_try"
saved_filepath = clf.save_model(saving_path_name)

In [ ]:
# define new model with basic parameters and load state dict weights

load_best = True

if load_best:
    saving_path_name = "./tabnet_model.zip"
else:
    saving_path_name = "./tabnet_model_test_second_try.zip"

loaded_clf = TabNetRegressor()
loaded_clf.load_model(saving_path_name)

In [ ]:
valid

In [ ]:
#out of time validation

preds_valid = loaded_clf.predict(valid.values)
valid2_mse = mean_squared_error(y_valid_all, preds_valid)


print(f"FINAL OUT OF TIME SCORE FOR {trainset_name} : {valid2_mse}")


In [ ]:
#Plot actual vs predicted in bins for validation set


plt.hist(preds_valid, 100, alpha=0.5, label='preds')
plt.hist(y_valid_all, 100, alpha=0.5, label='actual')
plt.legend(loc='upper right')
plt.show()

print(f"FINAL OUT OF TIME SCORE FOR {trainset_name} : {valid2_mse}")



In [ ]:
#transform back to original scale
preds_valid_orig = np.expm1(preds_valid)
y_valid_all_orig = np.expm1(y_valid_all)


plt.hist(preds_valid_orig, 100, alpha=0.5, label='preds')
plt.hist(y_valid_all_orig, 100, alpha=0.5, label='actual')
plt.legend(loc='upper right')
plt.show()

valid2_mse_orig = mean_squared_error(y_valid_all_orig, preds_valid_orig, squared=False)


print(f"Original Scale - FINAL OUT OF TIME SCORE FOR {trainset_name} : {valid2_mse_orig}")

In [ ]:
# if loaded_cl
preds = loaded_clf.predict(X_test)
test_mse = mean_squared_error(preds, y_test)
# else:
loaded_preds = clf.predict(X_test)
loaded_test_mse = mean_squared_error(loaded_preds, y_test)

print(f"FINAL TEST SCORE FOR {trainset_name} : {loaded_test_mse}")
print(f"FINAL TEST SCORE FOR {trainset_name} : {test_mse}")

# Global explainability : feat importance summing to 1

In [ ]:
clf.feature_importances_

# Local explainability and masks

In [ ]:
explain_matrix, masks = clf.explain(X_test)

In [ ]:
X_test.shape

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
explain_matrix.shape

In [ ]:
# for key, arr in masks.items():
#     masks[key] = np.sort(arr)


In [ ]:
explain_matrix_sorted = np.sort(explain_matrix, axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax.set_xticks([x for x in range(len(X_all.columns))])
ax.set_xticklabels([x for x in X_all.columns], rotation=90)

prev_val = explain_matrix_sorted[0][0] # set initial previous value
sampled_rows = [explain_matrix_sorted[0]] # set initial sampled row
for row in explain_matrix_sorted[1:]: # loop through remaining rows starting from the second row
    curr_val = row[0]
    if abs(curr_val - prev_val) > 0.005: # or any other threshold value you choose
        sampled_rows.append(row)
        if len(sampled_rows) == 100: # break out of loop after 100 rows have been sampled
            break
    prev_val = curr_val

ax.imshow(sampled_rows)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax.set_xticks([x for x in range(len(X_all.columns))])
ax.set_xticklabels([x for x in X_all.columns], rotation=90)
ax.imshow(explain_matrix[:50])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax.set_xticks([x for x in range(len(X_all.columns)+1)])
ax.set_xticklabels([x for x in X_all.columns] + ['Prediction'], rotation=90)
ax.imshow(np.append(explain_matrix[:50], y_test[:50].reshape(-1,1), axis=1))
plt.show()

In [ ]:
print(explain_matrix.shape)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,20))



for i in range(3):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")
    axs[i].set_xticks([x for x in range(len(X_all.columns))])
    axs[i].set_xticklabels([x for x in X_all.columns], rotation=90)

#Can we sort this? 
#By feature importance and then weight for mask? #TODO



In [ ]:
len([x for x in X_all.columns])

# XGB

In [ ]:
from xgboost import XGBRegressor

clf_xgb = XGBRegressor(max_depth=8,
    learning_rate=0.1,
    n_estimators=1000,
    verbosity=0,
    silent=None,
    objective='reg:linear',
    booster='gbtree',
    n_jobs=-1,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.7,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=0,
    seed=None,)

clf_xgb.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=40,
        verbose=10)

In [ ]:
preds = np.array(clf_xgb.predict(X_valid))
valid_mse = mean_squared_error(y_pred=preds, y_true=y_valid)
print(valid_mse)

preds = np.array(clf_xgb.predict(X_test))
test_mse = mean_squared_error(y_pred=preds, y_true=y_test)
print(test_mse)

In [ ]:
 #make a graph of the feature imporances including their names

feature_imp = pd.DataFrame(sorted(zip(clf_xgb.feature_importances_,X_all.columns)), columns=['Value','Feature'])

plt.figure(figsize=(18, 12))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('XGBoost Features')
plt.tight_layout()
plt.show()


In [ ]:
FINAL TEST SCORE FOR rossman-store-sales : 0.014900591809687539


In [ ]:
0.014900591809687539/0.00897729870324486